# Machine Learning Modelling on Datasets


We begin exploring Modelling for predicting crime in NYC using datasets.

---



In [ ]:
# load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import warnings
# warnings.filterwarnings('ignore')
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report


In [1]:
try:
  df = pd.read_csv('./ReadyData.csv')
  print("Data loaded successfully.")
except FileNotFoundError:
  print("Error: File not found. Please check the file path.")
except pd.errors.EmptyDataError:
  print("Error: The CSV file is empty.")
except pd.errors.ParserError:
  print("Error: There was an issue parsing the CSV file. Please ensure it's a valid CSV.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")

df

NameError: name 'pd' is not defined

In [ ]:
# df["Offense Description"].value_counts()

In [ ]:
# # stacked bar plt - Zara

# # Step 1: Count occurrences of each offense in each borough
# offense_boro_counts = df.groupby(['ARREST_BORO', 'Offense Description']).size().unstack(fill_value=0)

# # Use a colormap and get a list of distinct colors
# colors = plt.cm.get_cmap('tab20', len(offense_boro_counts.columns))

# # Step 3: Plot the data as a stacked bar plot with distinct colors
# offense_boro_counts.plot(kind='bar', stacked=True, figsize=(14, 7), color=[colors(i) for i in range(len(offense_boro_counts.columns))])

# # Add titles and labels
# plt.title('Offenses by Borough')
# plt.xlabel('Borough')
# plt.ylabel('Number of Offenses')
# plt.legend(title='Offense Description', bbox_to_anchor=(1.05, 1), loc='upper left')

# # Display the plot
# plt.tight_layout()  # Adjust layout to fit labels
# plt.show()

In [ ]:
# sns.pairplot(df, hue='longitute', 'latitude);

In [ ]:
# # Preprocessing: Extract temporal features
df['ARREST_DATE'] = pd.to_datetime(df['ARREST_DATE'])
df['DAY_OF_WEEK'] = df['ARREST_DATE'].dt.dayofweek
df['MONTH'] = df['ARREST_DATE'].dt.month

df

,ARREST_DATE,Offense Description,ARREST_BORO,Latitude,Longitude,ZONE,DAY_OF_WEEK,MONTH
0,2024-01-01,VEHICLE AND TRAFFIC LAWS,Brooklyn,40.651169,-73.941892,0,0,1
1,2024-01-01,DANGEROUS WEAPONS,Brooklyn,40.674123,-73.880803,7,0,1
2,2024-01-01,CRIMINAL MISCHIEF & RELATED OF,Bronx,40.834712,-73.919681,8,0,1
3,2024-01-01,FELONY ASSAULT,Brooklyn,40.621200,-73.995233,6,0,1
4,2024-01-01,CRIMINAL MISCHIEF & RELATED OF,Queens,40.756032,-73.828536,5,0,1
...,...,...,...,...,...,...,...,...
194307,2024-09-30,OFFENSES AGAINST PUBLIC ADMINI,Manhattan,40.768891,-73.982093,9,0,9
194308,2024-09-30,DANGEROUS DRUGS,Queens,40.693972,-73.794924,3,0,9
194309,2024-09-30,MISCELLANEOUS PENAL LAW,Brooklyn,40.698323,-73.917495,7,0,9
194310,2024-09-30,GRAND LARCENY,Queens,40.757691,-73.834115,5,0,9


In [ ]:
# Assigning crime severity scores for each type
crime_severity = {
    'MURDER & NON-NEGL. MANSLAUGHTER': 100,
    'RAPE': 100,
    'KIDNAPPING & RELATED OFFENSES': 100,
    'ARSON': 90,
    'FELONY ASSAULT': 90,
    'ROBBERY': 80,
    'DANGEROUS WEAPONS': 85,
    'GRAND LARCENY': 80,
    'CRIMINAL MISCHIEF & RELATED OF': 50,
    'VEHICLE AND TRAFFIC LAWS': 30,
    'ASSAULT 3 & RELATED OFFENSES': 60,
    'DANGEROUS DRUGS': 40,
    'SEX CRIMES': 95,
    'PETIT LARCENY': 40,
    'OTHER OFFENSES RELATED TO THEFT': 50,
    'FORGERY': 45,
    'OFFENSES INVOLVING FRAUD': 50,
    'OFFENSES AGAINST PUBLIC ADMINI': 40,
    'BURGLARY': 65,
    'POSSESSION OF STOLEN PROPERTY': 40,
    'INTOXICATED & IMPAIRED DRIVING': 35,
    'GRAND LARCENY OF MOTOR VEHICLE': 50,
    'CRIMINAL TRESPASS': 30,
    'OTHER STATE LAWS': 30,
    'MISCELLANEOUS PENAL LAW': 20,
    'DISORDERLY CONDUCT': 15,
    'PROSTITUTION & RELATED OFFENSES': 25,
    'CANNABIS RELATED OFFENSES': 10,
    'PARKING OFFENSES': 5,
    'LOITERING': 5,
    'HOMICIDE-NEGLIGENT,UNCLASSIFIE': 75,
    'THEFT-FRAUD': 50,
    'BURGLAR\'S TOOLS': 40,
    'ALCOHOLIC BEVERAGE CONTROL LAW': 15,
    'ADMINISTRATIVE CODE': 10,
}


df['CRIME_SEVERITY'] = df['Offense Description'].map(crime_severity)

df

,ARREST_DATE,Offense Description,ARREST_BORO,Latitude,Longitude,ZONE,DAY_OF_WEEK,MONTH,CRIME_SEVERITY
0,2024-01-01,VEHICLE AND TRAFFIC LAWS,Brooklyn,40.651169,-73.941892,0,0,1,30.0
1,2024-01-01,DANGEROUS WEAPONS,Brooklyn,40.674123,-73.880803,7,0,1,85.0
2,2024-01-01,CRIMINAL MISCHIEF & RELATED OF,Bronx,40.834712,-73.919681,8,0,1,50.0
3,2024-01-01,FELONY ASSAULT,Brooklyn,40.621200,-73.995233,6,0,1,90.0
4,2024-01-01,CRIMINAL MISCHIEF & RELATED OF,Queens,40.756032,-73.828536,5,0,1,50.0
...,...,...,...,...,...,...,...,...,...
194307,2024-09-30,OFFENSES AGAINST PUBLIC ADMINI,Manhattan,40.768891,-73.982093,9,0,9,40.0
194308,2024-09-30,DANGEROUS DRUGS,Queens,40.693972,-73.794924,3,0,9,40.0
194309,2024-09-30,MISCELLANEOUS PENAL LAW,Brooklyn,40.698323,-73.917495,7,0,9,20.0
194310,2024-09-30,GRAND LARCENY,Queens,40.757691,-73.834115,5,0,9,80.0


In [ ]:

# Handle location: Cluster latitude/longitude into zones
kmeans = KMeans(n_clusters=10, random_state=42)
df['ZONE'] = kmeans.fit_predict(df[['Latitude', 'Longitude']])

# Creating our target variable - Crime score
crime_score = df.groupby(['ZONE', 'DAY_OF_WEEK'])['CRIME_SEVERITY'].transform('mean')
df['CRIME_SCORE'] = crime_score

# Select features and target
X = df[['DAY_OF_WEEK', 'MONTH', 'Latitude', 'Longitude', 'ZONE']]
y = df['CRIME_SCORE']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize numerical features
scaler = StandardScaler()
X_train[['Latitude', 'Longitude']] = scaler.fit_transform(X_train[['Latitude', 'Longitude']])
X_test[['Latitude', 'Longitude']] = scaler.transform(X_test[['Latitude', 'Longitude']])

# Train a more suitable regression model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

Mean Squared Error: 0.00


In [24]:
# Cross-validation for better evaluation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated MSE: {-cv_scores.mean():.2f}')

# Predict crime score for a sample input
columns = ['DAY_OF_WEEK', 'MONTH', 'Latitude', 'Longitude', 'ZONE']
sample = [[1, 4, 40.750423, -73.98928, 4]]
sample_df = pd.DataFrame(sample, columns=columns)
sample_df[['Latitude', 'Longitude']] = scaler.transform(sample_df[['Latitude', 'Longitude']])
predicted_score = model.predict(sample_df)
print(f'Predicted Crime Score: {predicted_score[0]:.2f}')

Cross-validated MSE: 0.00
Predicted Crime Score: 57.10


In [26]:
# Cross-validation of 0.0 is insane 💀
df.CRIME_SCORE.describe()

,CRIME_SCORE
count,194312.000000
mean,55.063763
std,1.893177
min,30.000000
25%,53.655545
50%,55.015395
75%,56.188758
max,60.286230


In [ ]:
# df.groupby('ARREST_BORO').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
# plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# Handle location: Cluster latitude/longitude into zones
kmeans = KMeans(n_clusters=10, random_state=42)  # Adjust clusters based on dfset size
df['ZONE'] = kmeans.fit_predict(df[['Latitude', 'Longitude']])

# Creating our target variable - Crime score (Might change this to be safety score)
# Example: Mean severity score for the zone and day of the week
crime_score = df.groupby(['ZONE', 'DAY_OF_WEEK'])['CRIME_SEVERITY'].transform('mean')
df['CRIME_SCORE'] = crime_score

# Select features and target
X = df[['DAY_OF_WEEK', 'MONTH', 'Latitude', 'Longitude', 'ZONE']]
y = df['CRIME_SCORE']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize numerical features
scaler = StandardScaler()
X_train[['Latitude', 'Longitude']] = scaler.fit_transform(X_train[['Latitude', 'Longitude']])
X_test[['Latitude', 'Longitude']] = scaler.transform(X_test[['Latitude', 'Longitude']])

# Train a Gradient Boosting Regressor
model = GradientBoostingRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

Mean Squared Error: 0.15


In [ ]:
df

,ARREST_DATE,Offense Description,ARREST_BORO,Latitude,Longitude,DAY_OF_WEEK,MONTH,CRIME_SEVERITY,ZONE,CRIME_SCORE
0,2024-01-01,VEHICLE AND TRAFFIC LAWS,Brooklyn,40.651169,-73.941892,0,1,30.0,0,55.847271
1,2024-01-01,DANGEROUS WEAPONS,Brooklyn,40.674123,-73.880803,0,1,85.0,7,56.928080
2,2024-01-01,CRIMINAL MISCHIEF & RELATED OF,Bronx,40.834712,-73.919681,0,1,50.0,8,57.607588
3,2024-01-01,FELONY ASSAULT,Brooklyn,40.621200,-73.995233,0,1,90.0,6,53.258081
4,2024-01-01,CRIMINAL MISCHIEF & RELATED OF,Queens,40.756032,-73.828536,0,1,50.0,5,60.286230
...,...,...,...,...,...,...,...,...,...,...
194307,2024-09-30,OFFENSES AGAINST PUBLIC ADMINI,Manhattan,40.768891,-73.982093,0,9,40.0,9,53.257576
194308,2024-09-30,DANGEROUS DRUGS,Queens,40.693972,-73.794924,0,9,40.0,3,56.982204
194309,2024-09-30,MISCELLANEOUS PENAL LAW,Brooklyn,40.698323,-73.917495,0,9,20.0,7,56.928080
194310,2024-09-30,GRAND LARCENY,Queens,40.757691,-73.834115,0,9,80.0,5,60.286230


In [ ]:
# This is cooked
df.CRIME_SCORE.describe()

,CRIME_SCORE
count,194312.000000
mean,55.063763
std,1.893177
min,30.000000
25%,53.655545
50%,55.015395
75%,56.188758
max,60.286230


In [ ]:
# re-Define our feature names as they were during model training
columns = ['DAY_OF_WEEK', 'MONTH', 'Latitude', 'Longitude', 'ZONE']

# Sample data for example
sample = [[1, 4, 40.750423, -73.98928, 4]]

# Convert sample to a pandas DataFrame with feature names
sample_df = pd.DataFrame(sample, columns=columns)

# Normalize latitude and longitude (columns 'Latitude' and 'Longitude')
sample_df[['Latitude', 'Longitude']] = scaler.transform(sample_df[['Latitude', 'Longitude']])

# Predict crime score
predicted_score = model.predict(sample_df)
print(f'Predicted Crime Score: {predicted_score[0]:.2f}')


# # Example: Predict crime score for a specific zone on a specific date
# sample = [[2, 7, 40.750423, -73.98928, 3]]  # Day of week, Month, Lat, Long, Zone
# sample = np.array(sample)  # Convert to NumPy array

# # Normalize latitude and longitude
# sample[:, 2:4] = scaler.transform(sample[:, 2:4])  # Normalize the location features

# # Predict crime score
# predicted_score = model.predict(sample)
# print(f'Predicted Crime Score: {predicted_score[0]:.2f}')

Predicted Crime Score: 56.67


In [ ]:
label_encoder_offense = LabelEncoder()
label_encoder_boro = LabelEncoder()
df['Offense_Code'] = label_encoder_offense.fit_transform(df['Offense Description'])
df['ARREST_BORO_Code'] = label_encoder_boro.fit_transform(df['ARREST_BORO'])

# Features and target
X = df[['Offense_Code', 'Latitude', 'Longitude']]
y = df['ARREST_BORO_Code']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize numerical features
scaler = StandardScaler()
X_train[['Latitude', 'Longitude']] = scaler.fit_transform(X_train[['Latitude', 'Longitude']])
X_test[['Latitude', 'Longitude']] = scaler.transform(X_test[['Latitude', 'Longitude']])

# Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder_boro.classes_))
model

Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

        Bronx       1.00      1.00      1.00      8710
     Brooklyn       1.00      1.00      1.00     10783
    Manhattan       1.00      1.00      1.00      9179
       Queens       1.00      1.00      1.00      8507
Staten Island       1.00      1.00      1.00      1684

     accuracy                           1.00     38863
    macro avg       1.00      1.00      1.00     38863
 weighted avg       1.00      1.00      1.00     38863



# Trying to classification instead of regressions

In [28]:
df = pd.read_csv('./ReadyData.csv')
print("Data loaded successfully.")
# print(df.info())
# print(df.describe())

# Feature engineering
df['ARREST_DATE'] = pd.to_datetime(df['ARREST_DATE'])
df['DAY_OF_WEEK'] = df['ARREST_DATE'].dt.dayofweek
df['MONTH'] = df['ARREST_DATE'].dt.month
df['HOUR'] = df['ARREST_DATE'].dt.hour

Data loaded successfully.


In [30]:
# Handle location: Cluster latitude/longitude into zones
kmeans = KMeans(n_clusters=10, random_state=42)
df['ZONE'] = kmeans.fit_predict(df[['Latitude', 'Longitude']])

# Creating our target variable - Crime score
crime_score = df.groupby(['ZONE', 'DAY_OF_WEEK', 'HOUR'])['CRIME_SEVERITY'].transform('mean')
df['CRIME_SCORE'] = crime_score

# Define categories based on crime score
bins = [0, 20, 40, 60, 80, 100]
labels = ['Low crime chance', 'Lower medium crime chance', 'Medium crime chance', 'Higher medium crime chance', 'High crime chance']
df['CRIME_CATEGORY'] = pd.cut(df['CRIME_SCORE'], bins=bins, labels=labels)

# Select features and target
X = df[['DAY_OF_WEEK', 'MONTH', 'HOUR', 'Latitude', 'Longitude', 'ZONE']]
y = df['CRIME_CATEGORY']

In [31]:
# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Normalize numerical features
scaler = StandardScaler()
X_train[['Latitude', 'Longitude']] = scaler.fit_transform(X_train[['Latitude', 'Longitude']])
X_test[['Latitude', 'Longitude']] = scaler.transform(X_test[['Latitude', 'Longitude']])

# Train a classification model
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [35]:
# Predict and evaluate
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print("Classification Report:")
df

# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 1.00
Classification Report:


,ARREST_DATE,Offense Description,ARREST_BORO,Latitude,Longitude,DAY_OF_WEEK,MONTH,HOUR,CRIME_SEVERITY,ZONE,CRIME_SCORE,CRIME_CATEGORY
0,2024-01-01,VEHICLE AND TRAFFIC LAWS,Brooklyn,40.651169,-73.941892,0,1,0,30.0,0,55.847271,Medium crime chance
1,2024-01-01,DANGEROUS WEAPONS,Brooklyn,40.674123,-73.880803,0,1,0,85.0,7,56.928080,Medium crime chance
2,2024-01-01,CRIMINAL MISCHIEF & RELATED OF,Bronx,40.834712,-73.919681,0,1,0,50.0,8,57.607588,Medium crime chance
3,2024-01-01,FELONY ASSAULT,Brooklyn,40.621200,-73.995233,0,1,0,90.0,6,53.258081,Medium crime chance
4,2024-01-01,CRIMINAL MISCHIEF & RELATED OF,Queens,40.756032,-73.828536,0,1,0,50.0,5,60.286230,Higher medium crime chance
...,...,...,...,...,...,...,...,...,...,...,...,...
194307,2024-09-30,OFFENSES AGAINST PUBLIC ADMINI,Manhattan,40.768891,-73.982093,0,9,0,40.0,9,53.257576,Medium crime chance
194308,2024-09-30,DANGEROUS DRUGS,Queens,40.693972,-73.794924,0,9,0,40.0,3,56.982204,Medium crime chance
194309,2024-09-30,MISCELLANEOUS PENAL LAW,Brooklyn,40.698323,-73.917495,0,9,0,20.0,7,56.928080,Medium crime chance
194310,2024-09-30,GRAND LARCENY,Queens,40.757691,-73.834115,0,9,0,80.0,5,60.286230,Higher medium crime chance


In [37]:
# Lol expected this to not work but tried anyways :(
# Time to move on with the other dataset
df.CRIME_CATEGORY.value_counts()

,count
CRIME_CATEGORY,
Medium crime chance,192290
Higher medium crime chance,2021
Lower medium crime chance,1
Low crime chance,0
High crime chance,0
